In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from datetime import timedelta

In [2]:
data = pd.read_csv('trades.csv',sep=',')

In [3]:
data.head()

,MS US Equity,Unnamed: 1,Unnamed: 2,TXN US Equity,Unnamed: 4
0,Dates,Close,NaN,Dates,Close
1,23/1/2019 9:30,42.57,NaN,23/1/2019 9:30,96.63
2,23/1/2019 9:31,42.46,NaN,23/1/2019 9:31,96.42
3,23/1/2019 9:32,42.49,NaN,23/1/2019 9:32,96.51
4,23/1/2019 9:33,42.43,NaN,23/1/2019 9:33,96.81


In [4]:
formatDateTime = '%d/%m/%Y %H:%M'

In [5]:
#Settings

equity = 'TXN'

startTime = '9:20'
endTime = '16:00'
startDate = '7/6/2019'
endDate = '21/6/2019'

minutesDelta = 120

In [6]:
def securityIndex(equity, data):    
    ticker = equity + ' US Equity'
    index=0
    counter=0

    for col in data.columns:
        if col==ticker: 
            index = counter
        counter = counter+1
        
    return index    

In [7]:
index = securityIndex(equity, data)
print(index)

3


In [8]:
def processTimeSeries(startTime, startDate, endTime, endDate, equity, data, minutesDelta):

    array2D = []
    array2D.append([])
    array2D.append([])
 
    startDateTime = datetime.strptime(startDate+' '+startTime, formatDateTime)
    endDateTime = datetime.strptime(endDate+' '+endTime, formatDateTime)

    dateTime = startDateTime
    counter = 1
    index = securityIndex(equity, data)
    tradingDay = True
    incrementCounter = True

    while dateTime<=endDateTime:
        if incrementCounter or (recordDateTime<dateTime):
            counter = counter + 1
        if data.iloc[counter,index] != '' and data.iloc[counter,index] != 'nan':
            
            recordDateTime = datetime.strptime(data.iloc[counter,index], formatDateTime)
            #print(recordDateTime)
            if recordDateTime>=dateTime:
                minDelta = (recordDateTime-dateTime).days*24*60
                if recordDateTime == dateTime or (tradingDay==False and minDelta <=30.0):
                    array2D[0].append(dateTime)
                    dateTime = dateTime + timedelta(minutes=minutesDelta)
                    tradingDay = True 
                    incrementCounter = True
                    array2D[1].append(float(data.iloc[counter, index+1]))
                elif recordDateTime>dateTime:
                    eodDateTime = datetime.strptime(str(dateTime.day) +"/"+ str(dateTime.month) + "/"+ str(dateTime.year) +' '+endTime, formatDateTime)
                    if recordDateTime >= eodDateTime:
                        if tradingDay and (eodDateTime.day == recordDateTime.day):
                            array2D[0].append(eodDateTime)
                            array2D[1].append(float(data.iloc[counter, index+1]))
                            #print("Match2")
                        else:
                            incrementCounter = False
                        dateTime = dateTime + timedelta(days=1)
                        dateTime = datetime.strptime(str(dateTime.day) +"/"+ str(dateTime.month) + "/"+ str(dateTime.year) +' '+startTime, formatDateTime)
                    tradingDay = False
                    #print(dateTime)
                    #print(tradingDay)
                    #print((recordDateTime-dateTime).days*24*60)
                
    return array2D

In [9]:
processTimeSeries(startTime, startDate, endTime, endDate, equity, data, minutesDelta)

[[datetime.datetime(2019, 6, 7, 9, 20),
  datetime.datetime(2019, 6, 7, 11, 20),
  datetime.datetime(2019, 6, 7, 13, 20),
  datetime.datetime(2019, 6, 7, 15, 20),
  datetime.datetime(2019, 6, 10, 9, 20),
  datetime.datetime(2019, 6, 10, 11, 20),
  datetime.datetime(2019, 6, 10, 13, 20),
  datetime.datetime(2019, 6, 10, 15, 20),
  datetime.datetime(2019, 6, 11, 9, 20),
  datetime.datetime(2019, 6, 11, 11, 20),
  datetime.datetime(2019, 6, 11, 13, 20),
  datetime.datetime(2019, 6, 11, 15, 20),
  datetime.datetime(2019, 6, 12, 9, 20),
  datetime.datetime(2019, 6, 12, 11, 20),
  datetime.datetime(2019, 6, 12, 13, 20),
  datetime.datetime(2019, 6, 12, 15, 20),
  datetime.datetime(2019, 6, 13, 9, 20),
  datetime.datetime(2019, 6, 13, 11, 20),
  datetime.datetime(2019, 6, 13, 13, 20),
  datetime.datetime(2019, 6, 13, 15, 20),
  datetime.datetime(2019, 6, 14, 9, 20),
  datetime.datetime(2019, 6, 14, 11, 20),
  datetime.datetime(2019, 6, 14, 13, 20),
  datetime.datetime(2019, 6, 14, 15, 20),
  

In [55]:
#RSI

startTime = '16:00'
endTime = '16:00'
startDate = '1/2/2019'
endDate = '10/6/2019'

minutesDelta = 1440

eodTimeSeries = processTimeSeries(startTime, startDate, endTime, endDate, equity, data, minutesDelta)
pd.DataFrame(eodTimeSeries[1]).diff()

,0
0,NaN
1,-0.100
2,-0.090
3,0.520
4,4.160
5,-2.620
6,0.690
7,0.800
8,-0.590
9,1.680


In [66]:
def RSICalc(eodTimeSeries, period=14):
    delta = pd.DataFrame(eodTimeSeries[1]).diff()
    
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0

    # Calculate the SMA
    roll_up2 = up.rolling(window=period).mean()
    roll_down2 = (down.rolling(window=period).mean()).abs()
    
    # Calculate the RSI based on SMA
    RS2 = roll_up2 / roll_down2
    RSI2 = 100.0 - (100.0 / (1.0 + RS2))

    return RSI2

In [73]:
def MACD(eodTimeSeries, shortPeriod=12, longPeriod=26):

    price = pd.DataFrame(eodTimeSeries[1])
    exp1 = price.ewm(span=shortPeriod, adjust=False).mean()
    exp2 = price.ewm(span=longPeriod, adjust=False).mean()
    macd = exp1-exp2
    
    return macd

In [70]:
RSICalc(eodTimeSeries)

,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,NaN


In [74]:
MACD(eodTimeSeries)

,0
0,0.000000
1,-0.007977
2,-0.021316
3,0.009958
4,0.366199
5,0.432129
6,0.533902
7,0.671371
8,0.724359
9,0.891636
